# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel.

It makes use of the collection of native and semantic functions that have been registered to the kernel and using AI, will formulate a plan to execute the given ask.

From our own testing, planner works best with more powerful models like `gpt4` but sometimes you might get working plans with cheaper models like `gpt-35-turbo`. We encourage you to implement your own versions of the planner and use different models that fit your user needs.  

Read more about planner [here](https://aka.ms/sk/concepts/planner)

In [1]:
#!python -m pip install semantic-kernel==0.3.10.dev0

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI backend used by the kernel
if useAzureOpenAI:
    
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", AzureChatCompletion("gpt-35-turbo", endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_chat_service("gpt-3.5", OpenAIChatCompletion("gpt-3.5-turbo", api_key, org_id))

## It all begins with an ask

In [7]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas. She speaks French so write it in French.
Convert the text to uppercase"""

### Providing skills to the planner
The planner needs to know what skills are available to it. Here we'll give it access to the `SummarizeSkill` and `WriterSkill` we have defined on disk. This will include many semantic functions, of which the planner will intelligently choose a subset. 

You can also include native functions as well. Here we'll add the TextSkill.

In [4]:
from semantic_kernel.core_skills.text_skill import TextSkill

skills_directory = "../../samples/skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "SummarizeSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "WriterSkill")
text_skill = kernel.import_skill(TextSkill(), "TextSkill")

Define your ASK. What do you want the Kernel to do?

# Basic Planner

 Let's start by taking a look at a basic planner. The `BasicPlanner` produces a JSON-based plan that aims to solve the provided ask sequentially and evaluated in order.

In [5]:
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [8]:
basic_plan = await planner.create_plan_async(ask, kernel)

In [9]:
print(basic_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


You can see that the Planner took my ask and converted it into an JSON-based plan detailing how the AI would go about solving this task, making use of the skills that the Kernel has available to it.

As you can see in the above plan, the AI has determined which functions to call in order to fulfill the user ask. The output of each step of the plan becomes the input to the next function.

Let's also define an inline skill and have it be available to the Planner. Be sure to give it a function name and skill name.

In [10]:
sk_prompt = """
{{$input}}

Rewrite the above in the style of Shakespeare.
"""
shakespeareFunction = kernel.create_semantic_function(sk_prompt, "shakespeare", "ShakespeareSkill",
                                                      max_tokens=2000, temperature=0.8)

Let's update our ask using this new skill

In [20]:
ask = """
Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase."""

new_plan = await planner.create_plan_async(ask, kernel)

TypeError: SequentialPlanner.create_plan_async() takes 2 positional arguments but 3 were given

In [12]:
print(new_plan.generated_plan)

{
    "input": "Valentine's Day Date Ideas",
    "subtasks": [
        {"function": "WriterSkill.Brainstorm"},
        {"function": "ShakespeareSkill.shakespeare"},
        {"function": "WriterSkill.EmailTo", "args": {"recipient": "significant_other"}},
        {"function": "WriterSkill.Translate", "args": {"language": "French"}},
        {"function": "TextSkill.uppercase"}
    ]
}


### Executing the plan

Now that we have a plan, let's try to execute it! The Planner has a function called `execute_plan`.

In [13]:
results = await planner.execute_plan_async(new_plan, kernel)

In [14]:
print(results)

ASSUREZ-VOUS DE N'UTILISER QUE LE FRANÇAIS.

BONJOUR,

J'ESPÈRE QUE CET EMAIL VOUS TROUVE EN BONNE SANTÉ. JE VOULAIS PARTAGER AVEC VOUS QUELQUES IDÉES D'ACTIVITÉS QUE NOUS POUVONS APPRÉCIER ENSEMBLE.

1. QUE DIRIEZ-VOUS D'UN FESTIN ROMANTIQUE AUX CHANDELLES DANS LE CONFORT DE NOTRE FOYER ? NOUS POUVONS CRÉER UNE ATMOSPHÈRE CHALEUREUSE AVEC LA DOUCE LUEUR DES FLAMMES LUMINEUSES ET LAISSER NOS CŒURS S'ENFLAMMER ET NOS ÂMES S'ENTRELACER.

2. UNE AUTRE OPTION SERAIT DE FAIRE UN PIQUE-NIQUE DANS LE MAGNIFIQUE PARC. NOUS POURRIONS NOUS DÉTENDRE SUR UN TAPIS VERDOYANT ET NOUS RÉGALER DE METS DIGNES DES DIEUX, TOUT EN PROFITANT DE L'ÉTREINTE CHALEUREUSE DU SOLEIL.

3. SI VOUS PRÉFÉREZ ÊTRE CHOYÉ, NOUS POURRIONS NOUS ABANDONNER AUX CARESSES TENDRES DE MAINS EXPERTES DANS UN SANCTUAIRE BÉNI. NOUS POURRIONS ÊTRE PARÉS D'HUILES PARFUMÉES ET D'ÉLIXIRS APAISANTS, NOUS PERMETTANT DE VRAIMENT NOUS DÉTENDRE ET DE NOUS RESSOURCER.

4. QUE DIRIEZ-VOUS D'UNE VISITE AU VIGNOBLE ? NOUS POURRIONS PARTIR MAIN

# The Plan Object Model

To build more advanced planners, we need to introduce a proper Plan object that can contain all the necessary state and information needed for high quality plans.

To see what that object model is, look at (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/plan.py)

# Sequential Planner

The sequential planner is an XML-based step-by-step planner. You can see the prompt used for it here (https://github.com/microsoft/semantic-kernel/blob/main/python/semantic_kernel/planning/sequential_planner/Skills/SequentialPlanning/skprompt.txt)

In [15]:
from semantic_kernel.planning import SequentialPlanner
planner = SequentialPlanner(kernel)

In [21]:
print(ask)
sequential_plan = await planner.create_plan_async(goal=ask)


Tomorrow is Valentine's day. I need to come up with a few date ideas.
She likes Shakespeare so write using his style. She speaks French so write it in French.
Convert the text to uppercase.


To see the steps that the Sequential Planner will take, we can iterate over them and print their descriptions

In [22]:
for step in sequential_plan._steps:
    print(step.description, ":", step._state.__dict__)

Given a goal or topic description generate a list of ideas : {'variables': {'input': ''}}
Turn a scenario into a short and entertaining poem. : {'variables': {'input': ''}}
Generate a list of synopsis for a novel or novella with sub-chapters : {'variables': {'input': ''}}
Translate the input into a language of your choice : {'variables': {'input': ''}}
Generate an acronym for the given concept or phrase : {'variables': {'input': ''}}
Generate an acronym for the given concept or phrase : {'variables': {'input': ''}}
Generate an acronym for the given concept or phrase : {'variables': {'input': ''}}
Generate an acronym for the given concept or phrase : {'variables': {'input': ''}}
Convert a string to uppercase. : {'variables': {'input': ''}}


Let's ask the sequential planner to execute the plan.

In [23]:
result = await sequential_plan.invoke_async()

In [19]:
print(result)

Ideas for dates.


# Action Planner

The action planner takes in a list of functions and the goal, and outputs a **single** function to use that is appropriate to meet that goal.

In [24]:
from semantic_kernel.planning import ActionPlanner
planner = ActionPlanner(kernel)

Let's add more skills to the kernel

In [25]:
from semantic_kernel.core_skills import FileIOSkill, MathSkill, TextSkill, TimeSkill
kernel.import_skill(MathSkill(), "math")
kernel.import_skill(FileIOSkill(), "fileIO")
kernel.import_skill(TimeSkill(), "time")
kernel.import_skill(TextSkill(), "text")

{'lowercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1e858557950>,
 'trim': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1e858556490>,
 'trim_end': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1e8585568d0>,
 'trim_start': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1e858557890>,
 'uppercase': <semantic_kernel.orchestration.sk_function.SKFunction at 0x1e858556350>}

In [26]:
ask = "What is the sum of 110 and 990?"

In [27]:
plan = await planner.create_plan_async(goal=ask)

In [28]:
result = await plan.invoke_async()

In [29]:
print(result)

1100
